In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
url_train = './Dataset/development_dataset.csv' #traing dataset
url_lead = './Dataset/leaderboard_dataset.csv' #leaderboard dataset
url_dict = './Dataset/Data_Dictionary.xlsx' #data dictionary

In [3]:
## making dataframe

df_train = pd.read_csv(url_train)
df_lead = pd.read_csv(url_lead)
df_dict = pd.read_excel(url_dict)

In [4]:
df_train.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,...,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20,VAR21
0,1,828.235294,0.138889,321.428571,625.911006,1.818182,1.081550,198.113469,NaN,58.632548,...,100.083403,1.0,1,1.0,1.540594,NaN,1,0,100.104991,Low
1,2,911.764706,0.027778,NaN,611.574748,8.181818,1.344479,198.600020,22.086661,NaN,...,15.012510,NaN,1,NaN,1.614613,NaN,0,1,146.654045,High
2,3,962.352941,0.833333,35.714286,615.825381,8.181818,0.720796,197.267767,NaN,58.632548,...,210.175146,10.0,1,10.0,1.044599,NaN,0,0,98.249570,Medium
3,4,892.941177,NaN,NaN,638.076431,9.090909,0.820218,197.355744,4.363431,58.632548,...,NaN,NaN,1,NaN,1.145729,NaN,1,0,140.862306,Low
4,5,914.117647,0.083333,NaN,626.514988,5.181818,1.372928,198.790477,85.938202,58.632548,...,210.175146,NaN,.,NaN,1.558341,NaN,1,0,101.268503,High


In [5]:
df_lead.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20
0,1,878.823529,0.833333,821.428571,620.835806,5.181818,0.833974,197.500187,NaN,58.632548,9.433609,40.033361,10.000000,2,10.000000,1.212784,1000.00,1,0,48.099828
1,2,891.764706,0.138889,357.142857,614.092215,3.909091,0.595187,197.016843,253.896073,58.632548,10.144612,NaN,5.666667,1,7.600000,1.004907,992.55,0,0,104.018933
2,3,955.294118,0.055556,NaN,611.574748,6.363636,0.918652,197.660051,22.086661,NaN,0.583494,NaN,6.000000,1,6.000000,1.157115,NaN,1,1,120.586919
3,4,831.764706,0.111111,250.000000,617.740617,6.545455,2.383924,200.526288,NaN,48.539365,NaN,NaN,NaN,3,NaN,2.350858,NaN,0,1,168.592083
4,5,957.647059,0.166667,NaN,623.426802,NaN,0.711240,197.151458,5.013668,49.379748,0.255712,NaN,13.333333,1,13.333333,1.171551,NaN,0,1,109.204819


In [6]:
def get_percentage_missing(series):
    """ Calculates percentage of NaN values in DataFrame
    :param series: Pandas DataFrame object
    :return: float
    """
    num = series.isnull().sum()
    den = len(series)
    return round(num/den, 2)

In [7]:
df_with_any_null_values = df_train[df_train.columns[df_train.isnull().any()].tolist()]

lead_df_with_any_null_values = df_lead[df_lead.columns[df_lead.isnull().any()].tolist()]

In [8]:
get_percentage_missing(df_with_any_null_values)

VAR2     0.03
VAR3     0.10
VAR4     0.37
VAR5     0.00
VAR6     0.10
VAR7     0.02
VAR8     0.02
VAR9     0.47
VAR10    0.13
VAR11    0.22
VAR12    0.31
VAR13    0.25
VAR15    0.24
VAR16    0.03
VAR17    0.65
dtype: float64

In [9]:
get_percentage_missing(lead_df_with_any_null_values)

VAR2     0.03
VAR3     0.09
VAR4     0.37
VAR6     0.11
VAR7     0.02
VAR8     0.02
VAR9     0.47
VAR10    0.13
VAR11    0.22
VAR12    0.32
VAR13    0.25
VAR15    0.24
VAR16    0.02
VAR17    0.65
dtype: float64

In [10]:
# del df_train['VAR9']
# del df_train['VAR17']

# del df_lead['VAR9']
# del df_lead['VAR17']

In [11]:
df_train = df_train.replace(to_replace = '.', value =np.nan)

df_lead = df_lead.replace(to_replace = '.', value =np.nan)

In [12]:
#Required Imports

from sklearn.preprocessing import Imputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
#from sklearn.impute import SimpleImputer

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

# Techinques
# 1 - df.fillna()
# 2 - impute using sklearn - mean,meadian,mode
# 3 - interpolation 
# 4 - LDA analysis

In [13]:
itrImputetrain = IterativeImputer(max_iter=10, random_state=0)

In [14]:
# Imputing 
itrImputetrain = itrImputetrain.fit(df_train.iloc[:, 1:20].values)
df_train.iloc[:, 1:20] = itrImputetrain.fit_transform(df_train.iloc[:, 1:20].values)

C:\Users\PASHUPATI-PC\Anaconda3\envs\ml\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\PASHUPATI-PC\Anaconda3\envs\ml\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [15]:
df_lead.iloc[:,1:20].shape

(10000, 19)

In [16]:
itrImputelead = IterativeImputer(max_iter=10, random_state=0)

In [17]:
# Imputing 
itrImputelead = itrImputelead.fit(df_lead.iloc[:, 1:20].values)
df_lead.iloc[:, 1:20] = itrImputelead.fit_transform(df_lead.iloc[:, 1:20].values)

C:\Users\PASHUPATI-PC\Anaconda3\envs\ml\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\PASHUPATI-PC\Anaconda3\envs\ml\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [18]:
X = df_train.iloc[:, 1:-1].values
y = df_train.iloc[:, 20].values

In [19]:
y

array(['Low', 'High', 'Medium', ..., 'Low', 'Low', 'High'], dtype=object)

In [20]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X = LabelEncoder()
#X[:, 11] = labelencoder_X.fit_transform(X[:, 11])

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [21]:
y

array([1, 0, 2, ..., 1, 1, 0])

In [22]:
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.5493821912666621


In [23]:
n_df_with_any_null_values = df_train[df_train.columns[df_train.isnull().any()].tolist()]

In [24]:
get_percentage_missing(n_df_with_any_null_values)

Series([], dtype: float64)

In [25]:
df_lead.to_csv('itrImputedlead.csv')